In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


def get_sber_deposit_links():
    url = "https://www.sberbank.com/ru/person/contributions/deposits/prime"
    
    # Настройки браузера
    options = Options()
    # options.add_argument('--headless')  # включи при необходимости
    options.add_argument('--start-maximized')

    # Запуск драйвера
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 15)

    try:
        driver.get(url)

        # Пытаемся закрыть баннер с cookies (если есть)
        try:
            time.sleep(1)  # время на появление
            cookie_close = driver.find_element(By.CLASS_NAME, "kitt-cookie-warning__close")
            cookie_close.click()
            time.sleep(1)
        except:
            pass  # если нет — продолжаем

        # Ждём появления и кликаем по "Другие предложения по вкладам"
        accordion = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@data-test-id='AccordionSbolSummaryNew']")))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", accordion)
        driver.execute_script("arguments[0].click();", accordion)

        # Ждём появления раскрытого блока
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@data-test-id='AccordionSbolBodyNew']")))
        time.sleep(1)  # доп. ожидание подгрузки

        # Извлекаем все ссылки
        link_elements = driver.find_elements(By.XPATH, "//div[@data-test-id='AccordionSbolBodyNew']//a[@href]")
        deposit_links = set()

        for elem in link_elements:
            href = elem.get_attribute("href")
            if href and "/contributions/deposits" in href:
                deposit_links.add(href)

        return sorted(deposit_links)

    finally:
        driver.quit()


In [2]:
if __name__ == "__main__":
    links = get_sber_deposit_links()
    for link in links:
        print(link)


https://www.sberbank.com/ru/person/contributions/deposits/detskaya_kopilka
https://www.sberbank.com/ru/person/contributions/deposits/luchshiy_procent
https://www.sberbank.com/ru/person/contributions/deposits/luchshiy_procent/tablica_rascheta
https://www.sberbank.com/ru/person/contributions/deposits/nakopi
https://www.sberbank.com/ru/person/contributions/deposits/sbervklad_a
https://www.sberbank.com/ru/person/contributions/deposits/srochnyy-vklad
https://www.sberbank.com/ru/person/contributions/deposits/vklad-na-1-god
https://www.sberbank.com/ru/person/contributions/deposits/vklad-na-10-mesyacev
https://www.sberbank.com/ru/person/contributions/deposits/vklad-na-2-goda
https://www.sberbank.com/ru/person/contributions/deposits/vklad-na-2-mesyaca
https://www.sberbank.com/ru/person/contributions/deposits/vklad-na-3-goda
https://www.sberbank.com/ru/person/contributions/deposits/vklad-na-3-mesyaca
https://www.sberbank.com/ru/person/contributions/deposits/vklad-na-4-mesyaca
https://www.sberban

In [3]:
def parse_deposit_page(driver, url):
    driver.get(url)
    time.sleep(2)

    result = {
        "url": url,
        "title": None,
        "description": "",
    }

    try:
        # Пробуем вытащить заголовок (ищем h1 или заголовки внутри dk-sbol)
        try:
            title_elem = driver.find_element(By.TAG_NAME, "h1")
            result["title"] = title_elem.text.strip()
        except:
            try:
                title_elem = driver.find_element(By.CLASS_NAME, "dk-sbol-title")
                result["title"] = title_elem.text.strip()
            except:
                pass

        # Пробуем вытащить абзацы внутри TextWrapperTag
        paragraphs = driver.find_elements(By.XPATH, "//div[contains(@class, 'TextWrapperTag')]//p")
        result["description"] = "\n".join(p.text.strip() for p in paragraphs if p.text.strip())

    except Exception as e:
        result["error"] = str(e)

    return result


In [ ]:
if __name__ == "__main__":
    links = get_sber_deposit_links()

    # Повторно запускаем драйвер
    options = Options()
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=options)

    all_data = []

    for link in links:
        try:
            data = parse_deposit_page(driver, link)
            all_data.append(data)
            print(f"Parsed: {link}")
        except Exception as e:
            print(f"Error parsing {link}: {e}")

    driver.quit()

    # Печатаем JSON
    import json
    with open("sber_deposits.json", "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)

    print("✅ Done. Saved to sber_deposits.json")


Parsed: https://www.sberbank.com/ru/person/contributions/deposits/detskaya_kopilka
Parsed: https://www.sberbank.com/ru/person/contributions/deposits/luchshiy_procent
Parsed: https://www.sberbank.com/ru/person/contributions/deposits/luchshiy_procent/tablica_rascheta
Parsed: https://www.sberbank.com/ru/person/contributions/deposits/nakopi
Parsed: https://www.sberbank.com/ru/person/contributions/deposits/sbervklad_a
Parsed: https://www.sberbank.com/ru/person/contributions/deposits/srochnyy-vklad
Parsed: https://www.sberbank.com/ru/person/contributions/deposits/vklad-na-1-god
